# Régression linéaire et gradient stochastique

Ce TD-TP a pour objectif de plonger de manière un peu plus individualisée dans le cours d’optimisation stochastique. Il n’est pas noté, mais j’encourage très vivement les étudiants à rédiger consciencieusement leurs réponses et leurs idées. La rédaction force à mieux présenter les choses et surtout à mieux les cerner. Ca me permettra aussi plus facilement de corriger d’éventuelles incompréhensions.

## 1. Introduction

L’objectif de ce TP est d’illustrer la première partie du cours d’optimisation stochastique sur les erreurs en apprentissage et d’implémenter les premières versions du gradient stochastique. Il a aussi pour objectif de vous entraîner à effectuer des calculs avec des variables aléatoires pour les rendre plus accessibles et mieux comprendre les cours à venir.

Nous allons nous placer dans le cadre de travail le plus simple : la régression linéaire. Ce cadre présente plusieurs avantages :
- C’est probablement le plus simple d’un point de vue théorique et il permet d’appréhender de nombreux phénomènes avec des mathématiques relativement élémentaires.

- C’est probablement encore le plus utilisé dans les applications, et il me semble nécessaire de le comprendre profondément.

## 2. Le cadre
Soit $X$ un vecteur aléatoire de $\mathbb{R}^d$, pour $d \in \mathbb{N}$ suivant une certaine distribution de probabilité inconnue $P_X$. Pour un certain vecteur $\theta \in \mathbb{R}^d$, on construit une variable aléatoire $Y \in \mathbb{R}$ définie par :
$$
Y = \langle \theta, X\rangle + B \quad \text{ où } \quad B \sim \mathcal{N}(0, \sigma^2) \text{ est une variable aléatoire gaussienne indépendante de } X.
$$

> L’objectif de ce TP est d’apprendre le vecteur $\theta$ inconnu à partir de $n \in \mathbb{N}$ observations $(x_i, y_i)_{1 \leq i \leq n}$ tirées indépendamment suivant la loi $P$.

Pour ce faire, on peut simplement résoudre le problème de minimisation du risque empirique suivant :

\begin{equation}
    \underset{\omega \in \mathbb{R}^d}{inf} E_n(\omega) = \frac{1}{2n} \sum_{i = 1}^{n} (\langle \omega, x_i \rangle - y_i)^2
\end{equation}

On notera $\omega_n^*$ n’importe quel minimiseur (sous réserve d’existence) du problème ci-dessus.




## 6. Travail pratique (théorie)
Dans ce travail nous supposerons simplement que $X \sim \mathcal{N}(0, I_d)$. Nous supposons aussi que $\theta_i = 1$ pour tout $i$.

In [12]:
import numpy as np
import matplotlib.pyplot as plt

d = 10
n = 1000
theta = np.ones((d, 1))
sigma = 1

1. Définir une fonction \
$\texttt{X, Y = generate\_data(d,n,theta,sigma)}$

In [20]:
def generate_data(d, n, theta, sigma):
    # Vecteur X -> N(0, I_d)
    X = np.random.normal(0, 1, size=(n, d))

    # Variable gaussienne B
    B = np.random.normal(0, sigma, size=(n, 1))

    # Construction de Y : Y = <theta, X> + B
    Y = np.dot(X, theta) + B

    return X, Y

X, Y = generate_data(d, n, theta, sigma)

2. Définir une fonction qui renvoit le risque moyen \
$\texttt{E(w,theta,sigma)}$

In [14]:
def E(w, theta, sigma):
    return (1/2) * np.linalg.norm(w - theta)**2 + (sigma**2)/2

3. Définir une fonction qui renvoit le risque empirique \
$\texttt{En(w,X,Y)}$

In [15]:
def En(w, X, Y):
    n = len(Y)

    # Difference entre Xw et Y
    squared_diff = np.square(np.dot(X, w) - Y)

    # Risque empirique
    empirical_risk = 0.5 * np.sum(squared_diff) / n

    return empirical_risk

4. Définir une fonction qui renvoit le gradient du risque empirique \
$\texttt{grad\_En(w,X,Y)}$

In [26]:
def grad_En(w, X, Y):
    n = len(Y)

    # Différence entre Xw et Y
    diff = np.dot(X, w) - Y

    # Gradient du risque empirique
    gradient = (1/n) * np.dot(X.T, diff)

    return gradient

5. Définir une fonction qui renvoit le gradient stochastique suivant la loi uniforme \
$\texttt{grad\_sto\_En(w,X,Y,n\_batch)}$

In [17]:
def grad_sto_En(w, X, Y, n_batch):
    n = len(Y)

    # Choisir des indices aléatoires pour le batch
    batch_indices = np.random.choice(n, size=n_batch, replace=False)

    # Selectionner les données du batch
    X_batch = X[batch_indices, :]
    Y_batch = Y[batch_indices]

    # Calculer la différence entre Xw et Y
    diff_batch = np.dot(X_batch, w) - Y_batch

    # Calculer le gradient stochastique du risque empirique
    stochastic_gradient = (1/n_batch) * np.dot(X_batch.T, diff_batch)

    return stochastic_gradient

6. Calculer la constante de Lipschitz de $\nabla E_n$ numériquement.

In [18]:
def lipschitz_constant(X):
    n = len(X)

    # Constante de Lipschitz
    L = (1/n) * np.linalg.norm(np.dot(X.T, X))

    return L

print("Lipschitz constant : ", lipschitz_constant(X))

Lipschitz constant :  3.2184269609159117


7. Si vous aviez un choix, quelle méthode d’optimisation vous semblerait la plus efficace pour minimiser $E_n$ ?

Réponse question 7

8. Effectuer une descente de gradient à pas constant sur $E_n$ et stocker les suites $E_n(\omega_k)$ et $E(\omega_k)$.

In [35]:
def gradient_descent(X, Y, w_init, alpha, n_iterations):
    w = w_init
    En_history = []
    E_history = []

    for k in range(n_iterations):
        # Calculer le gradient du risque empirique
        gradient = grad_En(w, X, Y)

        # Mettre à jour le vecteur de poids
        w = w - alpha * gradient

        # Calculer et stocker le risque empirique
        En_k = En(w, X, Y)
        En_history.append(En_k)

        # Calculer et stocker l'erreur de prédiction
        E_k = E(w, theta, sigma)
        E_history.append(E_k)

    return En_history, E_history


alpha = 0.01
n_iterations = 100
w_init = np.random.rand(d, 1)

En_history, E_history = gradient_descent(X, Y, w_init, alpha, n_iterations)

9. Etudier l’erreur d’approximation $||\omega_n^* - \theta ||_2^2$ en fonction de $n$. Expliquez vos observations à partir du cours et des questions théoriques.

10. Effectuer un algorithme de gradient stochastique à pas constant sur $E_n$ et stocker les suites $E_n(\omega_k)$ et $E(\omega_k)$.

In [33]:
def stochastic_gradient_descent(X, Y, w_init, alpha, n_iterations):
    w = w_init
    n = len(Y)
    En_history = []
    E_history = []

    for k in range(n_iterations):
        # Choisir le nombre de données dans le batch
        n_batch = np.random.randint(1, n)

        # Calculer le gradient stochastique du risque empirique
        gradient = grad_sto_En(w, X, Y, n_batch)

        # Mettre à jour le vecteur de poids
        w = w - alpha * gradient

        # Calculer et stocker le risque empirique
        En_k = En(w, X, Y)
        En_history.append(En_k)

        # Calculer et stocker l'erreur de prédiction
        E_k = E(w, theta, sigma)
        E_history.append(E_k)

    return En_history, E_history

alpha = 0.01
n_iterations = 100
w_init = np.random.rand(d, 1)

En_history_stoc, E_history_stoc = stochastic_gradient_descent(X, Y, w_init, alpha, n_iterations)

11. Effectuer un algorithme de gradient stochastique à pas décroissant sur $E_n$ et stocker les suites $E_n(\omega_k)$ et $E(\omega_k)$.

In [34]:
def stochastic_gradient_descent_step(X, Y, w_init, n_iterations):
    w = w_init
    n = len(Y)
    En_history = []
    E_history = []

    for k in range(1, n_iterations + 1):
        # Choisir le nombre de données dans le batch
        n_batch = np.random.randint(1, n)

        # Calculer le gradient stochastique du risque empirique
        gradient = grad_sto_En(w, X, Y, n_batch)

        # Mettre à jour le vecteur de poids avec un pas de descente adaptatif
        alpha_k = 1 / np.sqrt(k)
        w = w - alpha_k * gradient

        # Calculer et stocker le risque empirique
        En_k = En(w, X, Y)
        En_history.append(En_k)

        # Calculer et stocker l'erreur de prédiction
        E_k = E(w, theta, sigma)
        E_history.append(E_k)

    return En_history, E_history


n_iterations = 100
w_init = np.random.rand(d, 1)

En_history_step, E_history_step = stochastic_gradient_descent_step(X, Y, w_init, n_iterations)

12. Comparer les taux de convergence pour le risque empirique et le risque moyen en fonction du nombre d’epoch pour chaque méthode.

13. Implémenter un algorithme de gradient stochastique online et comparer aux précédents.

14. Implémenter la méthode SAGA et comparer.

15. Relier les observations au cours. Quelle méthode devrait être privilégiée ?

16. Etudier l’influence de $n$, de $\sigma$.

## 7. Travail pratique (pratique)
Pour finir ce TP, nous proposons de tester les algorithmes de gradient stochastique dans un cadre moderne, avec la librairie PyTorch et des réseaux de Neurones. De très nombreux tutoriels sur ces logiciels existent et sont bien réalisés.

> Je vous propose ici de suivre le tutoriel suivant : [pytorch-mnist](https://nextjournal.com/gkoehler/pytorch-mnist).

Une fois que vous avez réussi à reproduire les expériences suggérées, comparez différents algorithmes d’optimisation disponibles sous PyTorch (SGD simple, SGD Momentum, RMSProp, SAGA, ADAM).

## 8. L’inégalité de Bernstein (inégalité de concentration)
$\textbf{Théorème 1 }$ (Inégalité de Bernstein). $\textit{Soient } Z_1, \dots , Z_n \textit{ un ensemble de } n \textit{ vecteurs aléatoires indépendants et identiquement distribués tels que } |Z_i| \leq c \textit{ presque sûrement, } \mathbb{E}(Z_i) = \mu \textit{ et } Var(Z_i) = \sigma^2 \textit{. Alors : }$

\begin{equation}
    \mathbb{P}(|\frac{1}{n}\sum_{i = 1}^n Z_i - \mu | \geq t) \leq 2 \exp(\frac{nt^2}{2 \sigma^2 + 2 ct /3}).
\end{equation}

Ce type d’inégalité est appelé inégalité de concentration. Il indique que la probabilité que la moyenne empirique dévie de la moyenne est très faible si on a un nombre d’observations suffisant.

1. Etablir la proposition suivante : l’inégalité suivante est valide avec une probabilité supérieure à $1 − \delta$.

2. On aimerait montrer que $E_n$ est proche de $E$. Comment utiliser l’inégalité de Bernstein ou le corollaire précédent ?